In [1]:
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfReader
import re
import unicodedata
import pandas as pd

In [2]:
def get_urls_de_ramas(url):

    urls=[]

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    contenedor = soup.find("div", id="carousel-links-gallery-parrafo")

    links = []
    if contenedor:
        for a in contenedor.find_all("a", href=True):
            links.append(a["href"])

    for link in links:
        urls.append(link)

    return urls


In [3]:
urls=get_urls_de_ramas("https://www.comunidad.madrid/servicios/educacion/oferta-grados-oficiales-universidades")
print(urls)

['https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-artes-humanidades', 'https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias-sociales-juridicas', 'https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias', 'https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias-salud', 'https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ingenieria-arquitectura']


In [4]:
def get_pdfs(urls):
    pdf_links = []

    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            fieldsets = soup.find_all("fieldset", class_="collapsible group-paragraph-parent-fieldset field-group-fieldset panel panel-default form-wrapper")

            for fieldset in fieldsets:
                for a in fieldset.find_all("a", href=True):
                    href = a["href"]
                    if href.lower().endswith(".pdf"):
                        if href.startswith("/"):
                            href = "https://www.comunidad.madrid" + href
                        pdf_links.append(href)

        except Exception as e:
            print(f"Error al procesar {url}: {e}")

        
    return pdf_links


In [5]:
pdfs=get_pdfs(urls)
print(pdfs)

['https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/01_antropologia_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/02_artes_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/03_diseno_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/04_estudios_culturales_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/05_estudios_de_asia_y_africa_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/06_estudios_semiticos_e_islamicos_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/07_filosofia_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/08_geografia_e_historia_2023_24_indiv.pdf', 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/09_humanidades_2023_24_indiv.pdf', 'https://www.comunidad.ma

In [6]:
def normalizar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'\s+', ' ', texto.strip())
    return texto

def extraer_area(texto):
    texto = normalizar_texto(texto)
    match = re.search(r'area de\s*(.*?)\s*rama de conocimiento', texto, re.IGNORECASE)
    return match.group(1).strip() if match else "(no encontrado)"

resultados = []

for url in pdfs:
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        pdf_file = io.BytesIO(response.content)
        reader = PdfReader(pdf_file)

        texto_total = ""
        for page in reader.pages[:1]: 
            texto_total += page.extract_text() or ""

        area = extraer_area(texto_total)

        resultados.append({
            "pdf_url": url,
            "area": area
        })

    except Exception as e:
        print(f"Error procesando {url}: {e}")
        resultados.append({
            "pdf_url": url,
            "area": "(error al procesar)"
        })

df_areas = pd.DataFrame(resultados)
df_areas.to_csv("df.csv", index=False)

In [7]:
df_areas

,pdf_url,area
0,https://www.comunidad.madrid/sites/default/fil...,antropologia social y cultural
1,https://www.comunidad.madrid/sites/default/fil...,artes
2,https://www.comunidad.madrid/sites/default/fil...,diseno
3,https://www.comunidad.madrid/sites/default/fil...,estudios culturales
4,https://www.comunidad.madrid/sites/default/fil...,estudios de asia y africa
...,...,...
70,https://www.comunidad.madrid/sites/default/fil...,la ciencia y la ingenieria de la energia y par...
71,https://www.comunidad.madrid/sites/default/fil...,ingenieria matematica
72,https://www.comunidad.madrid/sites/default/fil...,influencia de la ingenieria de mate - riales a...
73,https://www.comunidad.madrid/sites/default/fil...,ingenieria de telecomunicacion
